File computing the baseline performance of centralized algorithms from: https://www.nature.com/articles/s41531-022-00288-w

In [ ]:
from math import ceil

import pandas as pd

import numpy as np

import sys
import os
sys.path.append(os.path.abspath('..'))

In [ ]:
from multi_modality_fl.utils.data_management import BASE_PATH, DATASETS, build_full_path, drop_id, get_h5_data_keys, standardize_for_validation, normalize_classes

In [ ]:
dataset_path = build_full_path(BASE_PATH, DATASETS['combined_ppmi'])
print(dataset_path)
key = get_h5_data_keys(dataset_path)
dataset_full = pd.read_hdf(dataset_path, key=key[0])
display(drop_id(dataset_full))

In [ ]:
dataset_path = build_full_path(BASE_PATH, DATASETS['validation_pdbp'])
key = get_h5_data_keys(dataset_path)
print(dataset_path)
dataset_full_val = pd.read_hdf(dataset_path, key=key[0])

# balance the validation dataset
dataset_full_val = normalize_classes(dataset_full_val, label_column=['PHENO'], verbose=True)
display(drop_id(dataset_full_val))

In [ ]:
# validation_pdbp has 675 columns, but the combined ppmi dataset has 715. 
# The columns which are present in ppmi, but not pdbp are below:

ppmi_col = set(dataset_full.columns)
pdbp_col = set(dataset_full_val.columns)
print('ppmi', len(ppmi_col), 'pdbp', len(pdbp_col))
display(ppmi_col - pdbp_col)

# note, in the `display()` outputs above, the 'ID' property is removed prior to calling display.
# the number of columns may differ by 1 if the ID property is cleaned or not. 
print("Before normalization", dataset_full.shape, dataset_full_val.shape)
training_full, validation_full = standardize_for_validation(dataset_full, dataset_full_val)
print("After normalization", dataset_full.shape, dataset_full_val.shape)

In [ ]:
used_dataset_columns = list(ppmi_col & pdbp_col) # intersection
print(len(used_dataset_columns))

dataset_full = dataset_full[used_dataset_columns]
dataset_full_val = dataset_full_val[used_dataset_columns]

In [ ]:
# utils
import time
def utils_time_fn(fun, *args, **kwargs):
    """return (function run time (second), result of function call)"""
    start_time = time.perf_counter()
    
    result = fun(*args, **kwargs)

    end_time = time.perf_counter()
    run_time = end_time - start_time
    
    return (run_time, result)

def utils_sk_metrics_to_str(metrics_dict):
    """convert metrics dict to readable object"""
    rows = []
    for key, value in metrics_dict.items():
        if key == "algorithm":
            rows.append("{}: {}".format(key, value))
        elif key == "runtime_s":
            rows.append("{}: {:0.3f} seconds\n".format(key, value))
        else:
            rows.append("{}: {:0.4f}".format(key, value))
    return str.join("\n", rows)

In [ ]:
# extracted GenoML
import xgboost
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import neighbors
from sklearn import neural_network
from sklearn import svm

candidate_algorithms = [
    ensemble.AdaBoostRegressor(),
    ensemble.BaggingRegressor(),
    ensemble.GradientBoostingRegressor(),
    ensemble.RandomForestRegressor(n_estimators=10),
    linear_model.LinearRegression(),
    linear_model.SGDRegressor(),
    neighbors.KNeighborsRegressor(),
    neural_network.MLPRegressor(),
    svm.SVR(gamma='auto'),
    xgboost.XGBRegressor(),
    xgboost.XGBRFRegressor()
]

algorithms = {algorithm.__class__.__name__: algorithm for algorithm in candidate_algorithms}
print("\n".join(algorithms.keys()))

def evaluate(competing_metrics, algorithm_name, algorithm, x, y):
    """evaluate how an algorithm does on the provided dataset & generate a pd row"""
    run_time, pred = utils_time_fn(algorithm.predict, x)
    metric_results = [metric_func(y, pred) for metric_func in competing_metrics]
    
    row = [algorithm_name, run_time] + metric_results # + [TN, FN, TP, FP, sensitivity, specificity, PPV, NPV]
    return row

def process_results(column_names, results):
    log_table = pd.DataFrame(data=results, columns=column_names)
    best_id = log_table.explained_variance_score.idxmax()
    best_algorithm_name = log_table.iloc[best_id].algorithm
    best_algorithm = algorithms[best_algorithm_name]
    best_algorithm_metrics = log_table.iloc[best_id].to_dict()
    
    res = {
        'log_table': log_table,
        'best_id': best_id,
        'best_algorithm_name': best_algorithm_name,
        'best_algorithm': best_algorithm,
        'best_algorithm_metrics': best_algorithm_metrics,
    }
    
    return res

def compete(algorithms, x_train, y_train, x_test, y_test, x_addit_test=None, y_addit_test=None):
    """Compete the algorithms"""
    competing_metrics = [metrics.explained_variance_score, metrics.mean_squared_error,
                         metrics.median_absolute_error, metrics.r2_score, metrics.roc_auc_score]
    column_names = ["algorithm", "runtime_s"] + [metric.__name__ for metric in competing_metrics] # + ['TN', 'FN', 'TP', 'FP', 'sensitivity', 'specificity', 'PPV', 'NPV']

    results = []
    results_val = []
    for algorithm_name, algorithm in algorithms.items():

        algorithm.fit(x_train, y_train)
        
        row = evaluate(competing_metrics, algorithm_name, algorithm, x_test, y_test)
        results.append(row)
        
        row = evaluate(competing_metrics, algorithm_name, algorithm, x_addit_test, y_addit_test)
        results_val.append(row)
    
    res = process_results(column_names, results)
    results_val = process_results(column_names, results_val)
    
    return res, results_val

def get_split(dataset, splits):
    indeces = list(range(0, len(dataset)))
    np.random.shuffle(indeces)
    subsets = []
    for portion in splits:
        offset = 0
        if (subsets):
            offset = len(subsets[-1])

        indeces_partition = ceil(len(dataset) * portion)
        subset = dataset[offset: min(offset + indeces_partition, len(dataset))]
        subset = subset.reset_index(drop=True)
        # print(f"split {portion} - len: {len(subset)} actual: {len(subset) / len(dataset)}")
        # display(subset)
        subsets.append(subset)

    return subsets

In [ ]:
# one validation dataset for all repeats:
x_pdbp = dataset_full_val.drop(columns=['ID', 'PHENO'])
y_pdbp = dataset_full_val['PHENO']

repeats = 5
all_logs = []
all_logs_val = []
for fold in range(repeats):
    dataset_test, dataset_train = get_split(dataset_full, [0.15, 0.75])
    
    x_train = dataset_train.drop(columns=['ID', 'PHENO'])
    y_train = dataset_train['PHENO']
    x_test = dataset_test.drop(columns=['ID', 'PHENO'])
    y_test = dataset_test['PHENO']

    result, result_val = compete(algorithms, x_train, y_train, x_test, y_test, x_pdbp, y_pdbp)    
    
    all_logs.append(result)
    all_logs_val.append(result_val)
    
    print('='*100)
    print('\n', '-'*40, 'ppmi test set', '-'*40)
    display(result['log_table'])
    print(result['best_algorithm_name'])
    display(result['best_algorithm_metrics'])

    print('\n', '-'*40, 'pdbp validation set', '-'*40)
    display(result_val['log_table'])
    print(result_val['best_algorithm_name'])
    display(result_val['best_algorithm_metrics'])

In [ ]:
print(all_logs_val[0].keys())
print(all_logs_val[0]['best_algorithm_metrics'].keys())

In [ ]:
from typing import List, Dict

print(all_logs_val[0]['best_algorithm_metrics'].keys())
def get_relevant_metrics(metrics: List[str], metrics_dict: Dict):
    
    results = []
    for metric in metrics:
        results.append(f"{metric} {metrics_dict[metric]}")

    return " ".join(results)

print("\n".join([get_relevant_metrics(['algorithm', 'roc_auc_score'], alg_val_log['best_algorithm_metrics']) for alg_val_log in all_logs_val]))

In [ ]:
from collections import Counter

most_common_n = Counter([log['best_algorithm_name'] for log in all_logs_val]).most_common(1)
most_common, count = most_common_n[0]
print(most_common, count)

In [ ]:
best_algorithm = {most_common: algorithms[most_common]}

results = []
repeats = 5
for i in range(repeats):
    _, results_val = compete(best_algorithm, x_train, y_train, x_test, y_test, x_pdbp, y_pdbp)
    results.append(result_val)

In [ ]:
# the mean and std auc of the best performing algorithm
val_results = [result['best_algorithm_metrics']['roc_auc_score'] for result in results]
display(np.mean(val_results))
display(np.std(val_results))

### Prediction using best algorithm on PPMI Training dataset

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib as plt
alg = ensemble.GradientBoostingRegressor()

dataset_train_norm = normalize_classes(dataset_train, 'PHENO', verbose=True)
x_train_norm = dataset_train_norm.drop(columns=['ID', 'PHENO'])
y_train_norm = dataset_train_norm['PHENO']

fitted_alg = alg.fit(x_train, y_train)
y_pred = fitted_alg.predict(x_train_norm)

svc_disp = RocCurveDisplay.from_predictions(y_train_norm, y_pred)

### Prediction using best algorithm on holdout test PPMI Dataset (test set from training stage)

In [ ]:
alg = ensemble.GradientBoostingRegressor()
fitted_alg = alg.fit(x_train, y_train)
# balances predictor classes
dataset_test_norm = normalize_classes(dataset_test, 'PHENO', verbose=True)
x_test_norm = dataset_test.drop(columns=['ID', 'PHENO'])
y_test_norm = dataset_test['PHENO']

y_pred = fitted_alg.predict(x_test_norm)

# performance on holdout from ppmi dataset
svc_disp = RocCurveDisplay.from_predictions(y_test_norm, y_pred)

### Prediction using best algorithm on PDBP Dataset (validation)

In [ ]:
alg = ensemble.GradientBoostingRegressor()
fitted_alg = alg.fit(x_train, y_train)
y_pred = fitted_alg.predict(x_pdbp)

# performance on normalized 
svc_disp = RocCurveDisplay.from_predictions(y_pdbp, y_pred)
# plt.show()